In [ ]:
!pip install transformers[torch]

In [1]:
import numpy as np 
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [5]:
data_path = r""
data_csv = "C:/Users/mahad/Desktop/fyp2/data/written_name_train_v2.csv"
df = pd.read_csv(data_csv, names=['id','text']).iloc[1:]
df

,id,text
1,TRAIN_00001.jpg,BALTHAZAR
2,TRAIN_00002.jpg,SIMON
3,TRAIN_00003.jpg,BENES
4,TRAIN_00004.jpg,LA LOVE
5,TRAIN_00005.jpg,DAPHNE
...,...,...
330957,TRAIN_330957.jpg,LENNY
330958,TRAIN_330958.jpg,TIFFANY
330959,TRAIN_330959.jpg,COUTINHO DESA
330960,TRAIN_330960.jpg,MOURAD


In [6]:

df_train,df_test = train_test_split(df,test_size = 0.1,random_state=0)
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

sum([len(str(x).split()) for x in df_train['text']])/len(df_train)
max([len(str(x).split()) for x in df_train['text']])
sum([len(str(x).split()) for x in df_test['text']])/len(df_test)
df_test

,id,text
0,TRAIN_327782.jpg,FRANCOIS
1,TRAIN_114977.jpg,BAILLY
2,TRAIN_197059.jpg,CHOPLIN
3,TRAIN_313671.jpg,LUCAS
4,TRAIN_324514.jpg,FLAVIEN
...,...,...
33092,TRAIN_112186.jpg,HUGO
33093,TRAIN_233515.jpg,HELASSA
33094,TRAIN_270040.jpg,ILONA
33095,TRAIN_214091.jpg,OSCAR


In [7]:
from PIL import Image, ImageFilter, ImageEnhance, ImageDraw
import random
import numpy as np

def blur(img):
    return img.filter(ImageFilter.BLUR)

def gaussian_blur(img):
    return img.filter(ImageFilter.GaussianBlur(radius=random.randint(1, 10)))

def box_blur(img):
    return img.filter(ImageFilter.BoxBlur(radius=random.randint(1, 10)))

def contrast(img):
    enhancer = ImageEnhance.Contrast(img)
    return enhancer.enhance(random.uniform(0.5, 1.5))  # Adjust contrast level

def pixelate(img):
    return img.resize((256, 256)).resize(img.size, Image.NEAREST)

def rotate(img):
    return img.rotate(random.randint(-45, 45))  # Rotate randomly between -45 and 45 degrees

def perspective(img):
    width, height = img.size
    m = -0.5
    xshift = abs(m) * width
    new_width = width + int(round(xshift))
    return img.transform((new_width, height), Image.AFFINE, (1, m, -xshift if m > 0 else 0, 0, 1, 0), Image.BICUBIC)

def translate(img):
    return img.transform(img.size, Image.AFFINE, (1, 0, random.randint(-50, 50), 0, 1, random.randint(-50, 50)))

def noisy(noise_type, image):
    if noise_type == "gauss":
        mean = 0
        var = 0.1
        sigma = var ** 0.5
        gauss = np.random.normal(mean, sigma, image.size)
        noisy = np.array(image) + gauss
        return Image.fromarray(noisy.astype('uint8'))
    elif noise_type == "s&p":
        s_vs_p = 0.5
        amount = 0.004
        out = np.array(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.size]
        out[coords] = 255

        # Pepper mode
        num_pepper = np.ceil(amount * image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.size]
        out[coords] = 0
        return Image.fromarray(out.astype('uint8'))
    elif noise_type == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return Image.fromarray(noisy.astype('uint8'))
    elif noise_type == "speckle":
        gauss = np.random.randn(*image.size)
        noisy = np.array(image) + image * gauss
        return Image.fromarray(noisy.astype('uint8'))

def vertical_grid(img):
    draw = ImageDraw.Draw(img)
    line_width = random.randint(1, 3)
    for y in range(0, img.size[1], line_width + random.randint(1, 3)):
        draw.line([(0, y), (img.size[0], y)], fill='black', width=line_width)
    return img

def horizontal_grid(img):
    draw = ImageDraw.Draw(img)
    line_width = random.randint(1, 3)
    for x in range(0, img.size[0], line_width + random.randint(1, 3)):
        draw.line([(x, 0), (x, img.size[1])], fill='black', width=line_width)
    return img


In [8]:
import torch
from torch.utils.data import Dataset
from PIL import Image


def show_image(pathStr):
  img = Image.open(pathStr).convert("RGB")
  display(img)
  return img

def ocr_image(src_img):
  pixel_values = processor(images=src_img, return_tensors="pt").pixel_values
  generated_ids = model.generate(pixel_values)
  return processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
from transformers import ProcessorMixin, warnings

class CustomOCRProcessor(ProcessorMixin):
    # Define the attributes of the processor
    attributes = ["image_processor", "tokenizer"]

    # Define the default classes for image processor and tokenizer
    image_processor_class = "AutoImageProcessor"
    tokenizer_class = "AutoTokenizer"

    def __init__(self, image_processor=None, tokenizer=None, **kwargs):
        # Check if the 'feature_extractor' argument is used (deprecated)
        if "feature_extractor" in kwargs:
            warnings.warn(
                "The 'feature_extractor' argument is deprecated. Use 'image_processor' instead.",
                FutureWarning,
            )
            feature_extractor = kwargs.pop("feature_extractor")

        # Set the image processor and tokenizer
        image_processor = image_processor if image_processor is not None else feature_extractor
        if image_processor is None or tokenizer is None:
            raise ValueError("Both 'image_processor' and 'tokenizer' must be specified.")

        # Initialize the processor mixin
        super().__init__(image_processor, tokenizer)

    def __call__(self, *args, **kwargs):
        # Process either images or text based on input
        images = kwargs.pop("images", None)
        text = kwargs.pop("text", None)

        if images is not None:
            return self.image_processor(images, *args, **kwargs)  # Process images
        elif text is not None:
            return self.tokenizer(text, **kwargs)  # Process text
        else:
            raise ValueError("Either 'images' or 'text' input must be provided.")

    def batch_decode(self, *args, **kwargs):
        return self.tokenizer.batch_decode(*args, **kwargs)  # Batch decode tokens

    def decode(self, *args, **kwargs):
        return self.tokenizer.decode(*args, **kwargs)  # Decode tokens


In [ ]:
from transformers import TrOCRProcessor

image_processor = ViTImageProcessor.from_pretrained(
    'microsoft/swin-base-patch4-window12-384-in22k'
)
tokenizer = MBartTokenizer.from_pretrained(
    'facebook/mbart-large-50'
)
processor = CustomOCRProcessor(image_processor,tokenizer)
train_dataset = MedicalDataset(root_dir=data_path,
                           df=df_train,
                           processor=processor,max_target_length=55)
eval_dataset = MedicalDataset(root_dir=data_path,
                           df=df_test,
                           processor=processor,max_target_length=55)

In [ ]:
import os
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, Trainer, TrainingArguments

# Load the pre-trained model and processor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-handwritten") 
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-handwritten")


# Define a function to preprocess the images and texts
def preprocess_data(data):
    images = []
    texts = []
    for index, row in data.iterrows():
        image_path = os.path.join("C:/Users/mahad/Desktop/fyp2/data/train", row["FILENAME"])
        text = row["IDENTITY"]
        img = Image.open(image_path).convert("RGB")
        images.append(img)
        texts.append(text)
    return images, texts

train_images, train_texts = preprocess_data(df_train)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./trocr_fine_tuned",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
)

# Define Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=list(zip(train_images, train_texts)),
    # Add any other necessary arguments such as eval_dataset, data_collator, etc.
)

# Fine-tune the model
trainer.train()

# Evaluate the model (optional)
trainer.evaluate()